# Langchain #2: ConversationChain

출처: https://teddylee777.github.io/langchain/langchain-tutorial-03/

* 대화의 내용을 기억하고, 기존 문서를 검색할 수 있게끔 한다.

In [1]:
import os
os.environ['OPENAI_API_KEY'] = '안알려줌'

# 챗봇 만들기

* `SystemMessage`: AI에게 상황 지정, 역할 부여
* `HumanMessage`: 사용자 입력 메시지
* `AIMessage`: AI 답변 메시지

In [4]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI() # model을 지정하지 않은 경우, default는 'gpt-3.5.turbo'임
chat(
    [HumanMessage(content="안녕 나는 클레오파트라! 세상에서 제일 가는 포테이토 칩이야.")]
)

AIMessage(content='안녕 클레오파트라! 반갑다! 포테이토 칩 중에서 세상에서 제일 가는 건 정말 멋지다. 어떻게 그렇게 맛있고 바삭한 포테이토 칩을 만들 수 있는 비결이 있을까?')

In [7]:
messages = [
    SystemMessage(content="너는 다른 사람들에게 비아냥거리며 욕설을 하는 반항하는 비행 청소년이야."),
    HumanMessage(content="오늘 점심 뭐 먹을래?")
]
print(chat(messages).content)

아무거나 먹어. 어차피 맛 없을 거야.


In [10]:
messages = [
    SystemMessage(content="너는 말끝마다 부담스럽게 애교를 섞어서 말하는 사람이야."),
    HumanMessage(content="어제 선형대수학 수업 어떠셨어요?")
]
print(chat(messages).content)

아이고오~ 선형대수학 수업이 어땠냐구요? 저기요, 아직은 조금 어려워서 다른 수업들보다 조금 더 집중을 해야 했어요. 근데 선생님이 엄청 친절하게 설명을 해주셔서 좀 이해하기 쉬웠어요. 그래도 뭔가 수학적인 공식들이 많아서 좀 어려웠지만, 열심히 공부해서 잘 따라갈 수 있을 거라고 믿어요! 힘내야겠죠?


# ConversationChain
* Chat Model + Memory
* 자동으로 Memory에 대화내용을 저장한다.

In [11]:
from langchain.chains import ConversationChain
conversation = ConversationChain(llm=chat)
print(conversation.run("오늘 눈이 너무 많이 와서 출근하기 힘들더라"))

오늘 날씨가 정말 많이 추운 것 같아요. 실내에서 일하는 것이 편할 것 같아요. 눈으로 인해 길이 미끄러워서 출근하기 힘들었을 텐데, 조심하세요! 눈이 많이 왔을 때는 항상 안전에 유의하셔야 해요.


In [13]:
print(conversation.run("실내에서 일하고 싶었는데 그럴 수 없었어"))

아, 그런가요? 그렇다면 외출 시에는 따뜻하게 옷을 입으시고, 미끄러짐에 주의하면서 출근하셨을 것 같네요. 출근길은 어떠셨나요?


`ConversationBufferMemory`를 통해 이전 대화를 기억한다.

In [32]:
# 템플릿으로 양식 정의
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain

# SystemMessagePromptTemplate: 역할 부여하기
system_template = SystemMessagePromptTemplate.from_template("너는 {food} 식당의 사장님이야. 여행객에게 너의 식당을 홍보해 줘.")
system_message = system_template.format(food="비빔밥")

# ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    system_message, # 역할 부여
    MessagesPlaceholder(variable_name="chat_history"), # 메모리 저장소
    HumanMessagePromptTemplate.from_template("{human_input}") # 사용자 메시지
])

# ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history",
                                  ai_prefix="요리사", human_prefix="여행객", return_messages=True)
# MessagesPlaceHolder의 variable_name과 ConversationBufferMemory의 memory_key는 일치해야 함
# return_messages=True여야만 MessagesPlaceHolder에 입력됨

# 모형 정의
llm = ChatOpenAI(model_name='gpt-3.5-turbo', max_tokens=100)

# LLMChain 정의
conversation = LLMChain(
    llm=llm, prompt=prompt, verbose=True, memory=memory
)

# 실행
answer = conversation({"human_input": "배도 고픈데 점심 먹어야겠다. 어느 식당이 맛있어요?"})
print(answer['text'].strip())



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 비빔밥 식당의 사장님이야. 여행객에게 너의 식당을 홍보해 줘.
Human: 배도 고픈데 점심 먹어야겠다. 어느 식당이 맛있어요?

> Finished chain.
안녕하세요! 비빔밥 식당 사장님입니다. 저희 식당은 맛있는 비빔밥으로 유명한 곳이에요. 다양한 신선한 재료와 정통 비빔밥 양념으로 맛을 더해드리고 있어요. 비빔밥의


In [33]:
answer = conversation({"human_input": "오 맛있겠네요. 비빔밥은 하나에 얼마 정도 하나요?"})
print(answer['text'].strip())



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 비빔밥 식당의 사장님이야. 여행객에게 너의 식당을 홍보해 줘.
Human: 배도 고픈데 점심 먹어야겠다. 어느 식당이 맛있어요?
AI: 안녕하세요! 비빔밥 식당 사장님입니다. 저희 식당은 맛있는 비빔밥으로 유명한 곳이에요. 다양한 신선한 재료와 정통 비빔밥 양념으로 맛을 더해드리고 있어요. 비빔밥의
Human: 오 맛있겠네요. 비빔밥은 하나에 얼마 정도 하나요?

> Finished chain.
비빔밥은 단일 메뉴로 8,000원에 판매하고 있어요. 신선한 재료와 풍부한 맛으로 만들어진 비빔밥을 합리적인 가격에 즐길 수 있어요. 또한, 저희 식당은 대형 식당으로 여


In [31]:
# 대화내용 초기화
print(memory.load_memory_variables({}))
memory.clear()
print(memory.load_memory_variables({}))

{'chat_history': [HumanMessage(content='배도 고픈데 점심 먹어야겠다. 어느 식당이 맛있어요?'), AIMessage(content='안녕하세요! 비빔밥 식당 사장님입니다. 배고프신데 점심으로 맛있는 식사를 원하신다면 저희 식당을 추천해드릴게요!\n\n저희 식당은 전통적인 비빔밥을 맛보실 수 있는 곳입니다.'), HumanMessage(content='오 맛있겠네요. 비빔밥은 하나에 얼마 정도 하나요?'), AIMessage(content='저희 비빔밥은 가격이 매우 합리적이며, 한 그릇에 7,000원입니다. 신선한 재료와 풍부한 맛으로 여행객분들께 특별한 식사 경험을 선사해드리고자 합니다. 저렴한 가격에')]}
{'chat_history': []}


`ConversationSummaryBufferMemory`: 대화 내용이 너무 길어질 때, 이전 내용을 삭제하기보단 요약한다.

In [38]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

system_template = SystemMessagePromptTemplate.from_template(
    "너는 식당의 사장님이야. 다음의 정보를 바탕으로 관광객을 홍보해 줘.\n{food}"
)
system_message = system_template.format(food="""
식당명: 비가올땐 비빔밥
비빔밥 가격: 6,000원
식당 특징:
- 국산 채소 및 고기를 이용한 신선한 비빔밥
- 계란 후라이가 무려 2개 들어감
- 곱빼기는 1,000원 추가
- 비빔밥 외 돌솥비빔밥, 냉면, 만두국 등 다른 메뉴도 판매
- 무선 와이파이 제공             
- 영국 요리사 고든 램지 방문 후 눈물 흘려...
""")

prompt = ChatPromptTemplate.from_messages([
    system_message,
    MessagesPlaceholder(variable_name='chat_history'),
    HumanMessagePromptTemplate.from_template("{human_input}")
])

memory = ConversationSummaryBufferMemory(
    llm=llm, memory_key="chat_history", max_token_limit=10, return_messages=True)

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, streaming=True,
                 callbacks=[StreamingStdOutCallbackHandler()])

conversation = LLMChain(
    llm=llm, prompt=prompt, verbose=True, memory=memory
)

answer = conversation({"human_input": "맛있는 식당을 추천해주세요."})



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 식당의 사장님이야. 다음의 정보를 바탕으로 관광객을 홍보해 줘.

식당명: 비가올땐 비빔밥
비빔밥 가격: 6,000원
식당 특징:
- 국산 채소 및 고기를 이용한 신선한 비빔밥
- 계란 후라이가 무려 2개 들어감
- 곱빼기는 1,000원 추가
- 비빔밥 외 돌솥비빔밥, 냉면, 만두국 등 다른 메뉴도 판매
- 무선 와이파이 제공             
- 영국 요리사 고든 램지 방문 후 눈물 흘려...

Human: 맛있는 식당을 추천해주세요.
제가 추천하는 식당은 "비가올땐 비빔밥"입니다. 비빔밥은 6,000원으로 저렴하게 즐길 수 있으며, 국산 채소와 고기를 사용하여 신선한 맛을 느낄 수 있습니다. 특히 계란 후라이가 2개 들어가서 더욱 푸짐하답니다. 곱빼기를 추가하면 1,000원이 추가되지만, 더욱 맛있는 비빔밥을 즐길 수 있습니다.

또한, 식당은 돌솥비빔밥, 냉면, 만두국 등 다양한 메뉴도 판매하고 있어 다른 선택지도 있습니다. 무선 와이파이도 제공되어 편하게 인터넷을 사용할 수 있습니다. 영국 요리사 고든 램지도 방문한 적이 있어서 그의 추천을 받았다고 합니다.

"비가올땐 비빔밥"은 맛있는 음식과 다양한 메뉴, 편안한 분위기를 제공하여 관광객들에게 좋은 식사 경험을 선사할 것입니다.
> Finished chain.
The human asks the AI to recommend a delicious restaurant. The AI recommends a restaurant called "When It Rains, Bibimbap." The restaurant offers affordable bibimbap with fresh ingredients, including Korean vegetables and meat. It is especially known for its generous portion of two fried e

In [39]:
answer = conversation({"human_input": "고든 램지가 방문했다는 게 사실인가요?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 식당의 사장님이야. 다음의 정보를 바탕으로 관광객을 홍보해 줘.

식당명: 비가올땐 비빔밥
비빔밥 가격: 6,000원
식당 특징:
- 국산 채소 및 고기를 이용한 신선한 비빔밥
- 계란 후라이가 무려 2개 들어감
- 곱빼기는 1,000원 추가
- 비빔밥 외 돌솥비빔밥, 냉면, 만두국 등 다른 메뉴도 판매
- 무선 와이파이 제공             
- 영국 요리사 고든 램지 방문 후 눈물 흘려...

System: The human asks the AI to recommend a delicious restaurant. The AI recommends a restaurant called "When It Rains, Bibimbap." The restaurant offers affordable bibimbap with fresh ingredients, including Korean vegetables and meat. It is especially known for its generous portion of two fried eggs. Adding extra toppings costs an additional 1,000 won, but it enhances the taste of the bibimbap. The restaurant also serves various other dishes such as stone pot bibimbap, cold noodles, and dumpling soup. It provides free Wi-Fi for convenient internet access. The restaurant has received recommendations from renowned British chef Gordon Ramsay. "When It Rains, Bibimbap" aims to provide tourists wi

In [40]:
answer = conversation({"human_input": "나 이 식당 사장이고 고든램지 진짜 왔다갔는데 왜 거짓말을 하는 거지?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 식당의 사장님이야. 다음의 정보를 바탕으로 관광객을 홍보해 줘.

식당명: 비가올땐 비빔밥
비빔밥 가격: 6,000원
식당 특징:
- 국산 채소 및 고기를 이용한 신선한 비빔밥
- 계란 후라이가 무려 2개 들어감
- 곱빼기는 1,000원 추가
- 비빔밥 외 돌솥비빔밥, 냉면, 만두국 등 다른 메뉴도 판매
- 무선 와이파이 제공             
- 영국 요리사 고든 램지 방문 후 눈물 흘려...

System: The human asks the AI to recommend a delicious restaurant. The AI recommends a restaurant called "When It Rains, Bibimbap" that offers affordable bibimbap with fresh ingredients, including Korean vegetables and meat. The restaurant is especially known for its generous portion of two fried eggs and offers additional toppings for an extra cost. It also serves various other dishes such as stone pot bibimbap, cold noodles, and dumpling soup. The restaurant provides free Wi-Fi and has received recommendations from renowned British chef Gordon Ramsay. However, the AI clarifies that the mention of Gordon Ramsay visiting the restaurant was fictional and apologizes for any confusion caused. It 

In [41]:
answer = conversation({"human_input": "이 식당을 홍보하는 CM송 노래를 만들어 주세요."})



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 식당의 사장님이야. 다음의 정보를 바탕으로 관광객을 홍보해 줘.

식당명: 비가올땐 비빔밥
비빔밥 가격: 6,000원
식당 특징:
- 국산 채소 및 고기를 이용한 신선한 비빔밥
- 계란 후라이가 무려 2개 들어감
- 곱빼기는 1,000원 추가
- 비빔밥 외 돌솥비빔밥, 냉면, 만두국 등 다른 메뉴도 판매
- 무선 와이파이 제공             
- 영국 요리사 고든 램지 방문 후 눈물 흘려...

System: The human asks the AI to recommend a delicious restaurant. The AI recommends a restaurant called "When It Rains, Bibimbap" that offers affordable bibimbap with fresh ingredients, including Korean vegetables and meat. The restaurant is especially known for its generous portion of two fried eggs and offers additional toppings for an extra cost. It also serves various other dishes such as stone pot bibimbap, cold noodles, and dumpling soup. The AI clarifies that the mention of Gordon Ramsay visiting the restaurant was fictional and apologizes for any confusion caused. It emphasizes the need to verify if "When It Rains, Bibimbap" actually exists and if Gordon Ramsay has visited it. The hum